In [24]:
library(Seurat)
library(SeuratWrappers)
library(Matrix)
library(glmGamPoi)
library(sctransform)
library(zinbwave)
library(dplyr)

In [17]:
exp_data <- read.csv('../../data/muraro/matrix.csv', row.names=1)
exp_data_meta <- read.csv('../../data/muraro/meta.csv', row.names=1)[1:2285,]
colnames(exp_data) <- rownames(exp_data_meta)
size_factor <- exp_data_meta$nCount_RNA/mean(exp_data_meta$nCount_RNA)
pbmc <- CreateSeuratObject(exp_data)
pbmc <- NormalizeData(pbmc)
pbmc <- FindVariableFeatures(pbmc, selection.method='vst', nfeatures=2000)
pbmc <- ScaleData(pbmc, features = rownames(pbmc))

Centering and scaling data matrix



In [18]:
donor <- read.csv('/data01/hanbin973/batch/batch_labels.csv')$X0
pbmc@meta.data$donor <- donor[1:2285]
pbmc@meta.data$logsf <- log(size_factor)

# PCA

In [7]:
pbmc_pca <- RunPCA(pbmc)

PC_ 1 
Positive:  IFITM3, ZFP36L1, REST, MYH9, S100A11, SOX4, NOTCH2, MYL12A, RBPMS, LGALS3 
	   PMEPA1, AHNAK, CAV2, TMSB4X, YAP1, EPS8, CLIC4, PPIC, ANO6, TMSB10 
	   TPM1, CDC42EP1, SH3BP4, MSN, RHOC, LITAF, CSDA, LAMC1, ANXA2, OSMR 
Negative:  SCG3, SYT7, PAM, ERO1LB, ABCC8, GAD2, VGF, MIR7-3HG, PLCXD3, LOC728392 
	   KIF5C, MLXIPL, STX1A, G6PC2, STMN2, PEG10, CFC1, RGS4, SCD, WNT4 
	   RASD1, C14orf132, ELAVL4, PRUNE2, KCNK16, SYT13, PCP4, CRYBA2, ATP2A3, MEIS2 
PC_ 2 
Positive:  CD24, ELF3, TACSTD2, KRT8, KIAA1522, KRT18, CFB, ANXA4, OCLN, KRT7 
	   CLDN4, SERPINA3, TMC5, RASEF, PDZK1IP1, CLDN1, TC2N, ABCC3, SDC4, TMPRSS2 
	   ERBB3, CXADR, LAD1, TM4SF1, CLMN, GATM, LCN2, KCNQ1, ATP10B, NR5A2 
Negative:  SPARC, PDGFRB, COL6A3, MRC2, COL5A2, COL5A1, CDH11, NID1, COL5A3, COL1A2 
	   LAMA4, COL15A1, COL6A2, COL3A1, FBN1, BGN, LOXL2, THBS2, LUM, LTBP2 
	   SFRP2, PRRX1, CYGB, VCAN, SRPX2, WNT5A, EDNRA, ITGA11, COL12A1, COL4A1 
PC_ 3 
Positive:  CFTR, ONECUT2, DCDC2, TINAGL1, ALDH1A3,

In [8]:
write.csv(Embeddings(pbmc_pca), 'pca.csv')

# SCTransform

In [9]:
pbmc_sct <- SCTransform(pbmc, batch_var='donor', method='glmGamPoi')

Calculating cell attributes from input UMI matrix: log_umi

Variance stabilizing transformation of count matrix of size 17138 by 2285

Model formula is y ~ (log_umi) : donor + donor + 0

Get Negative Binomial regression parameters per gene

Using 2000 genes, 2285 cells



  |======================================================================| 100%


Found 88 outliers - those will be ignored in fitting/regularization step


Some genes not detected in batch D28 -- assuming a low mean.

Some genes not detected in batch D29 -- assuming a low mean.

Some genes not detected in batch D30 -- assuming a low mean.

Some genes not detected in batch D31 -- assuming a low mean.

Second step: Get residuals using fitted parameters for 17138 genes



  |======================================================================| 100%


Computing corrected count matrix for 17138 genes



  |======================================================================| 100%


Calculating gene attributes

Wall clock passed: Time difference of 1.135273 mins

Determine variable features

Place corrected count matrix in counts slot

Centering data matrix

Set default assay to SCT



In [10]:
pbmc_sct <- RunPCA(pbmc_sct)
write.csv(Embeddings(pbmc_sct), 'sct.csv')

PC_ 1 
Positive:  CHGB, CHGA, CPE, SCG5, SCG2, PCSK2, SLC30A8, VGF, TM4SF4, TTR 
	   GCG, GC, ALDH1A1, SCGN, G6PC2, MAFB, PAX6, NEUROD1, BEX1, PPP1R1A 
	   RGS4, SCG3, PTPRN, ABCC8, ENPP2, SLC7A2, CRYBA2, LOXL4, TMEM176B, PAM 
Negative:  CD24, TACSTD2, ZFP36L1, IFITM3, TMSB4X, CD44, ANXA4, SOX4, SERPINA3, ANXA2 
	   SAT1, TM4SF1, CFB, KRT7, TPM1, S100A11, GDF15, PMEPA1, CAV2, CLDN1 
	   SOD2, SLC4A4, KRT8, KRT18, NFIB, CFTR, BACE2, CXCL2, MYH9, GSTA1 
PC_ 2 
Positive:  CD24, SERPINA3, SPINK1, REG1A, CFB, TACSTD2, GSTA1, CTRB2, REG1B, PRSS1 
	   CPA1, ANXA4, PRSS3P2, GSTA2, PLA2G1B, PNLIP, OLFM4, CPA2, DUOX2, CPB1 
	   MUC1, C15orf48, CTRB1, GDF15, SAT1, TM4SF1, ANPEP, ALDOB, LYZ, REG3A 
Negative:  SPARC, COL4A1, PXDN, COL4A2, UNC5B, IGFBP4, COL1A2, LGALS1, COL5A2, SERPINH1 
	   LAMA4, TIMP1, COL5A1, COL6A1, COL15A1, COL1A1, COL6A2, COL3A1, NID1, PDGFRB 
	   COL6A3, SERPINE1, FSTL1, CALD1, MRC2, FN1, COL5A3, LOXL2, NOTCH3, ITGA1 
PC_ 3 
Positive:  TM4SF4, GC, GCG, PCSK2, TTR, CHGB, CFTR

# GLM-PCA

In [11]:
pbmc_gpca <- RunGLMPCA(pbmc, L=30, 
                       X=as.matrix(model.matrix(as.formula('~factor(donor)'), pbmc@meta.data)), 
                       sz=size_factor)

Sparse matrices are not supported for minibatch='none'. Coercing to dense matrix. If this exhausts memory, consider setting minibatch to 'stochastic' or 'memoized'.



In [12]:
write.csv(Embeddings(pbmc_gpca, reduction='glmpca'), 'glmpca.csv')

# ZINB-WaVE

In [26]:
sceobj <- as.SingleCellExperiment(pbmc)
filter1 <- rowSums(assay(sceobj)>5)>5
sceobj <- sceobj[filter1,]
assay(sceobj) %>% log1p %>% rowVars -> vars
names(vars) <- rownames(sceobj)
vars <- sort(vars, decreasing = TRUE)
sceobj <- sceobj[names(vars)[1:2000],]
assay(sceobj) <- apply(assay(sceobj), c(1,2), function(x){
        as.integer(x)
})
sceobj <- zinbsurf(sceobj, K=20, X='~factor(donor)+logsf')

In [35]:
reducedDim(sceobj)

ERROR: Error in .get_internal_missing(x, basefun = reducedDim, namefun = reducedDimNames, : no available entries for 'reducedDim(<SingleCellExperiment>, ...)'


# ARI & Silhouette

In [13]:
library(mclust)
library(cluster)

Package 'mclust' version 5.4.7
Type 'citation("mclust")' for citing this R package in publications.



In [38]:
read.csv('zinbwave.csv', row.names=1)

,W1,W2,W3,W4,W5,W6,W7,W8,W9,W10,W11,W12,W13,W14,W15,W16,W17,W18,W19,W20
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
D28-1_74,-0.50820137,0.2435890799,-0.26909431,-0.11983702,-0.05026566,0.2019553,0.0156414161,-0.4374301,-0.183061066,0.34225854,-0.14703743,-0.01202278,0.33423608,0.383609421,-0.185955788,0.101153232,0.062629276,-0.368599781,-0.33348810,0.157241298
D28-1_76,-0.10567818,0.0073923083,0.16904180,-0.13123328,-0.58547493,0.3481552,0.2048372595,-0.9981003,-0.214695373,0.27775234,0.29206372,0.04979500,-0.08372900,0.036148923,0.184385852,-0.039192137,0.073883610,-0.439519741,0.14536439,0.282282596
D28-1_85,-0.26924521,0.1311500537,0.19121525,0.11845643,-0.47096231,0.5271673,0.2850241232,-0.8017929,-0.449482472,0.09912014,0.13246777,-0.05781881,0.05063047,0.157321139,-0.101868891,-0.153917928,0.304793089,-0.128855352,-0.30734504,0.089628999
D28-2_37,-0.68806720,0.2208983777,0.06427102,0.05406720,0.19812520,0.5184167,0.5054173390,-0.4713570,-0.339456875,0.21266277,-0.09039184,-0.12720322,-0.41147388,0.757909458,-0.369971680,0.155162745,0.038561655,-0.493026143,-0.51900182,0.054819264
D28-2_63,-0.29576195,0.0005054661,0.01475363,0.14166750,-0.34688440,0.8600988,0.9914231290,-0.9355535,-0.018688357,-0.14131127,0.48036902,-0.19599447,0.33499029,0.655914662,0.146390656,-0.591363663,0.404757004,0.586206478,0.54176044,-0.035438176
D28-2_91,-0.39414122,-0.0713654686,0.04022291,-0.07969127,-0.24980921,0.6115871,0.0709196145,-0.7987031,-0.096207161,0.02859287,-0.11571020,-0.18678731,0.22170538,-0.064070145,-0.352459520,-0.129695008,0.066302092,-0.267336951,0.45823529,0.113909847
D28-3_14,-0.05017596,-0.0215278076,0.06727658,-0.11152558,-0.42103536,0.4201850,0.4726066034,-0.6969464,-0.214273733,0.05793723,0.12960212,0.04659955,0.41421153,0.495734355,0.108653169,-0.206819583,0.126031105,-0.002112414,-0.10403772,-0.025863380
D28-3_18,-0.17651452,-0.0995936007,0.13657419,-0.24909924,-0.48658332,0.4150801,-0.1438963569,-0.6897620,-0.481102143,0.11876012,-0.14337744,-0.24349009,0.34607631,-0.118798799,-0.036847760,-0.146433400,0.114866684,-0.299291129,-0.09914388,-0.009173711
D28-4_26,-0.13592117,0.0345372296,0.26900642,-0.07586915,-0.45437461,0.4074102,0.3697353120,-0.7241819,-0.693471619,-0.17584711,0.35945101,-0.09739673,0.21884291,-0.241430149,0.149107656,-0.193896444,-0.081362618,0.127949046,-0.19374713,-0.094203142


In [39]:
pca <- read.csv('pca.csv', row.names=1)
sct <- read.csv('sct.csv', row.names=1)
glmpca <- read.csv('glmpca.csv', row.names=1)
rdpca <- read.csv('rdpca.csv', header=FALSE)
rownames(rdpca) <- rownames(pca)
zinbwave <- read.csv('zinbwave.csv', row.names=1)

pbmc[['pca']] <- CreateDimReducObject(embeddings = as.matrix(pca), key = "pc_", assay='RNA')
pbmc[['sct']] <- CreateDimReducObject(embeddings = as.matrix(sct), key = "sct_", assay='RNA')
pbmc[['glmpca']] <- CreateDimReducObject(embeddings = as.matrix(glmpca), key = "glmpc_", assay='RNA')
pbmc[['rdpca']] <- CreateDimReducObject(embeddings = as.matrix(rdpca), key = "rdpc_", assay='RNA')
pbmc[['zinbwave']] <- CreateDimReducObject(embeddings = as.matrix(zinbwave), key = "zinbwave_", assay='RNA')

In [43]:
celltype <- exp_data_meta$celltype
intname <- 1:length(unique(celltype))
names(intname) <- unique(celltype)
for (method in c('pca', 'sct', 'glmpca', 'rdpca', 'zinbwave')){
    obj <- FindNeighbors(pbmc, reduction=method, dims=1:30, verbose=FALSE)
    obj <- FindClusters(obj, resolution=1.5, verbose=FALSE)
    
    print(method)
    print(length(unique(obj$seurat_clusters)))
    print(adjustedRandIndex(obj$seurat_clusters, celltype))
}

for (method in c('pca', 'sct', 'glmpca', 'rdpca', 'zinbwave')){
    obj <- RunTSNE(pbmc, reduction=method, perplexity=70, dims=1:20)
    dist_mat <- dist(Embeddings(obj, reduction='tsne'))
    silscore <- silhouette(as.vector(intname[celltype]), dist_mat) 
    
    print(method)
    print(summary(silscore)$avg.width)

}

[1] "pca"
[1] 15
[1] 0.4911688
[1] "sct"
[1] 20
[1] 0.415895
[1] "glmpca"
[1] 28
[1] 0.06228432
[1] "rdpca"
[1] 13
[1] 0.6041042
[1] "zinbwave"
[1] 14
[1] 0.5310524
[1] "pca"
[1] 0.4905845
[1] "sct"
[1] 0.4138554
[1] "glmpca"
[1] -0.1121223
[1] "rdpca"
[1] 0.5883029
[1] "zinbwave"
[1] 0.4511044
